In [88]:
import pandas
import numpy as np
import urllib
import googlemaps
import json
import time, datetime
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
from string import punctuation
from bs4 import BeautifulSoup
import math
import pickle

In [2]:
panda_data = pandas.read_json('train.json')

In [3]:
with open('records.json', 'w') as outfile:
    json.dump(json.loads(panda_data.to_json(orient='records')), outfile)

In [4]:
def parseData(fname):
    for l in urllib.urlopen(fname):
        yield eval(l)
        
data = np.transpose(list(parseData('records.json')))

In [56]:
for i, k in enumerate(data[0][0]):
    print i+1, '.', k, ':', data[0][0][k]


1 . display_address : Metropolitan Avenue
2 . description : A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy These Following Apartment Features As You Rent Here? Modern Designed Bathroom w/ a Deep Spa Soaking Tub? Room to Room AC/Heat? Real Oak Hardwood Floors? Rain Forest Shower Head? SS steel Appliances w/ Chef Gas Cook Oven & LG Fridge? washer /dryer in the apt? Cable Internet Ready? Granite Counter Top Kitchen w/ lot of cabinet storage spaceIt's Just A Few blocks To L Train<br /><br />Don't miss out!<br /><br />We have several great apartments in the immediate area.<br /><br />For additional information 687-878-2229<p><a  website_redacted 
3 . price : 3000
4 . bedrooms : 3
5 . photos : ['https://photos.renthop.com/2/7211212_1ed4542ec81621d70d1061aa833e669c.jpg', 'https://photos.renthop.com/2/7211212_7dfc41dced69245065df83d08eed4a00.jpg', 'https://photos.renthop.com/2/7211212_c17853c4b869af6f53af08b0f5820b4c.jpg', 'https://photos.renthop.com/2/7211212_787ad8ea0c089792e7453e2121f8ac89.j

In [55]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text, stemmer=PorterStemmer()):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')

def trainTfIdfModel(strings):
    content = np.array([])
    tfIdfScores = np.array([])
    print 'Preprocessing Data'
    for s in strings:
        t = " ".join(item.strip() for item in BeautifulSoup(s).find_all(text=True))
        t = str(''.join([i if ord(i) < 128 else ' ' for i in t]))
        replace_punctuation = string.maketrans(punctuation, ' '*len(punctuation))
        transformed_string = t.lower().translate(replace_punctuation)
        content = np.append(content, transformed_string)
     
    print 'Started Training'
    tfs = tfidf.fit_transform(content)

trainTfIdfModel([d[0]['description'] for d in data])

def getTransformedtfIdfScore(review):
    t = " ".join(item.strip() for item in BeautifulSoup(review).find_all(text=True))
    t = str(''.join([i if ord(i) < 128 else ' ' for i in t]))
    replace_punctuation = string.maketrans(punctuation, ' '*len(punctuation))
    transformed_string = t.lower().translate(replace_punctuation)
    response = tfidf.transform([transformed_string])
    
#     feature_names = tfidf.get_feature_names()
#     for col in response.nonzero()[1]:
#         print feature_names[col], ' - ', response[0, col]
    
#     print response.nonzero()[1]
    if float(response.nonzero()[1].shape[0]) == 0:
        return 0
    return float(np.sum([response[0, col] for col in response.nonzero()[1]])) / float(response.nonzero()[1].shape[0])

/Users/Saransh/anaconda/envs/tensorflow_env/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


In [84]:
def getARI(string):
    string = " ".join(item.strip() for item in BeautifulSoup(string).find_all(text=True))
    string = str(''.join([i if ord(i) < 128 else ' ' for i in string]))
    s_count = float(len(nltk.sent_tokenize(string)))
    w_count = float(len(nltk.wordpunct_tokenize(string)))
    c_count = float(len(string))
    
    if s_count == 0:
        return 12
    
    return math.ceil((4.71*(c_count/w_count)) + (0.5*(w_count/s_count)) - 21.43)



def GetFeatures(data):
    structured_data = []
    index = 1
    for d in data:
        feat = []
        if index % 10000 == 1:
            print 'Starting', index
            
        feat.append(time.mktime(datetime.datetime.strptime(d[0]['created'], "%Y-%m-%d %H:%M:%S").timetuple()))
        feat.append(getTransformedtfIdfScore(d[0]['description']))
        feat.append(getARI(d[0]['description']))
        structured_data.append(feat)
        if index % 10000 == 1:
            print 'Done', index
        index += 1
    
    return structured_data

In [85]:
new_data = GetFeatures(data)

Starting 1
Done 1
Starting 10001
Done 10001
Starting 20001
Done 20001
Starting 30001
Done 30001
Starting 40001
Done 40001


In [86]:
print new_data[0]
print np.array(new_data).shape, np.array(new_labels).shape

[1466780064.0, 0.10791517320475999, 6.0]
(49352, 3) (49352,)


In [93]:
print new_data[:10]

[[1466780064.0, 0.10791517320475999, 6.0], [1465759167.0, 0, 12], [1460888801.0, 0.11633887623912888, 12.0], [1460971322.0, 0.15148662481922268, 11.0], [1461832361.0, 0.13460773829788475, 11.0], [1461065087.0, 0, 12], [1461752396.0, 0.11315999966405803, 17.0], [1460552502.0, 0.09805834665654639, 48.0], [1461144995.0, 0, 12], [1459591095.0, 0.10279767184574277, 14.0]]


In [87]:
np.save('ut_tfidf_ari', new_data)

In [91]:
import pickle
with open('interest_prediction/data', 'r') as dump_file:
    dhruv_data = pickle.load(dump_file)

ValueError: insecure string pickle